Let's add the rest of the methods that we need to get the behavior we saw in the demonstration.  Notice that we can call other methods of a class from inside methods by accessing them through `self`.

In [ ]:
import numpy as np

class Forest(object):
    def __init__(self, size=(150, 150), p_sapling=0.0025, p_lightning=5.0e-6,
                 name=None):
        self.size = size
        self.trees = np.zeros(self.size, dtype=bool)
        self.fires = np.zeros(self.size, dtype=bool)
        self.p_sapling = p_sapling
        self.p_lightning = p_lightning
        if name is not None:
            self.name = name
        else:
            self.name = self.__class__.__name__

    @property
    def num_cells(self):
        return self.size[0] * self.size[1]

    @property
    def fire_fraction(self):
        return self.fires.sum() / float(self.num_cells)

    @property
    def tree_fraction(self):
        return self.trees.sum() / float(self.num_cells)

    def advance_one_step(self):
        self.grow_trees()
        self.start_fires()
        self.burn_trees()

    def burn_trees(self):
        """The rules for burning trees:
        -New fires start in tree-filled adjacent cells.
        -Trees in currently burning cells burn and disappear.

        The fire-spreading algorithm is calculated on an array padded
        around the edges with zeros to allow vectorized computations.
        """
        working_size = (self.size[0] + 2, self.size[1] + 2)
        fires = np.zeros(working_size, dtype=bool)
        fires[1:-1, 1:-1] = self.fires
        north = fires[:-2, 1:-1]
        south = fires[2:, 1:-1]
        east = fires[1:-1, :-2]
        west = fires[1:-1, 2:]
        new_fires = (north | south | east | west) & self.trees
        self.trees[self.fires] = False
        self.fires = new_fires

    def grow_trees(self):
        growth_sites = self._rand_bool(self.p_sapling)
        self.trees[growth_sites] = True

    def start_fires(self):
        lightning_strikes = self._rand_bool(self.p_lightning) & self.trees
        self.fires[lightning_strikes] = True

    def _rand_bool(self, p):
        """ A private method that will be reused by other methods. """
        return np.random.uniform(size=self.trees.shape) < p

We can test that we have the basic operations in place by creating an instance of `Forest` and calling some of its methods:

In [ ]:
forest = Forest()
forest.grow_trees()
forest.start_fires()
forest.burn_trees()
print forest.tree_fraction

Now let's use our new class to create two different forests, one that burns rarely, and another that has more frequent lightning strikes.

In [ ]:
low_fire_forest = Forest(p_lightning=5e-6, name="Infrequent fires")
high_fire_forest = Forest(p_lightning=1e-4, name="Frequent fires")
tree_counts = []
for i in range(2500):
    low_fire_forest.advance_one_step()
    high_fire_forest.advance_one_step()
    tree_counts.append((low_fire_forest.tree_fraction, high_fire_forest.tree_fraction))

A little plotting help from `matplotlib` lets us take a look at the results.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure(figsize=(9,4))
ax0 = fig.add_subplot(1,3,1)
ax0.imshow(low_fire_forest.trees, 
           interpolation="none",
           origin='lower', cmap=plt.cm.Greens)
ax0.set_title(low_fire_forest.name)

ax1 = fig.add_subplot(1,3,2)
ax1.plot(tree_counts)
ax1.legend(['less-frequent', 'more-frequent'], loc='best')

ax2 = fig.add_subplot(1,3,3)
ax2.imshow(high_fire_forest.trees, 
           interpolation="none",
           origin='lower', cmap=plt.cm.Greens)
ax2.set_title(high_fire_forest.name)

We see a remarkable difference in the behavior of the two forests.  The forest with frequent fires has a relatively stable poulation of trees, while the forest with less frequent fires shows dramatic oscillations as the forest grows and is burned back by larger fires.

Copyright 2008-2016, Enthought, Inc.<br>Use only permitted under license.  Copying, sharing, redistributing or other unauthorized use strictly prohibited.<br>http://www.enthought.com